# 1880-2010年间全美婴儿姓名  
pandas dataframe pd.read_csv()  
python 字符串插值 数字  
pandas dataframe pd.concat()  
pandas dataframe d.pivot_table()  
pandas dataframe d.tail()  
pandas dataframe d.plot()  
pandas DataFrameGroupBy d.apply(func)  
pandas dataframe d.xxx  
pandas series s.astype()  
pandas series s.sum()

In [ ]:
import pandas as pd
names1880 = pd.read_csv('ch02/yob1880.txt',names=['name','sex','births'],header=None)
names1880[:10]

In [ ]:
names1880.info()

In [ ]:
#获取按sex分组统计
names1880.groupby('sex').births.sum()

In [ ]:
#将各个年份的数据合并
years = range(1880,2011)

pieces = []
columns = ['name','sex','births']

for year in years:
    path = 'ch02/yob%d.txt' % year
    frame = pd.read_csv(path,names=columns,header=None)
    
    frame['year']=year
    pieces.append(frame)

#将所有数据整合到单个DataFrame中
names = pd.concat(pieces,ignore_index=True)
names.info()

In [ ]:
total_births = names.pivot_table(values ='births',columns='sex',index='year',aggfunc=sum)
total_births.tail()

In [ ]:
total_births.plot(kind='line',title='Total births by sex and year')

In [ ]:
def add_prop(group):
    #用于插入一个prop列，存放指定名字的婴儿数相对于总出生数的比例
    births = group.births.astype(float)
    group['prop'] = births/births.sum()
    return group

names = names.groupby(['year','sex']).apply(add_prop)
names.info()